## Chapter 2-6, 2강 임베딩 기반 유사도 검색 & 미니 RAG (CPU)

- 목표: 문서 임베딩으로 top‑k 검색 구현, 간단 템플릿 요약으로 미니 RAG 구성
- 데이터: 짧은 문서 코퍼스(뉴스 제목/요약), 2k 정도 샘플


### 구성 (Overview)
- 코퍼스 로드/정제 → SBERT 임베딩 생성/캐시 → 최근접 탐색(코사인) → 미니 RAG 응답 조합 → 간단 데모


### 0-1. 1강(텍스트 분류) ↔ 2강(임베딩·RAG) 연계 개요
- 1강 요지: 전통 ML(TF‑IDF+LR) vs 문장 임베딩(SBERT+LR) 비교, 속도/성능 트레이드오프 확인
- 2강 초점: SBERT 임베딩을 활용한 유사도 기반 검색과 미니 RAG 조합
- 브리지 포인트:
  - TF‑IDF: 단어 빈도 기반(빠름, 실시간/대량 처리 유리), 의미적 유사도 한계
  - SBERT: 문맥 의미 기반(정확도/일반화 장점), CPU 단독 시 속도/추론비용 부담
  - 실무: 하이브리드(스파스 TF‑IDF + 덴스 SBERT) 혹은 단계적 파이프라인이 효과적


### 0. 환경 설정 및 라이브러리


In [10]:
# -*- coding: utf-8 -*-
import os, time, random, numpy as np, pandas as pd, matplotlib.pyplot as plt
from typing import Tuple

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

from sentence_transformers import SentenceTransformer, util
from sklearn.neighbors import NearestNeighbors

# 시드 고정
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
set_seed(42)


### 1. 코퍼스 준비
- `datasets`에서 뉴스 샘플 로드(없으면 간단 더미 데이터)
- 텍스트 길이 필터, 중복 제거


In [23]:
from datasets import load_dataset

def load_news_corpus(n_per_class: int = 500) -> pd.DataFrame:
    """
    AG News 데이터셋에서 클래스별로 동일한 수의 샘플을 추출하여 로드합니다.

    Args:
        n_per_class (int): 각 클래스에서 가져올 샘플 수 (기본=500)

    Returns:
        df (pd.DataFrame): 뉴스 데이터 프레임
            - text  : 뉴스 기사 텍스트
            - label : 카테고리 (0=World, 1=Sports, 2=Business, 3=Sci/Tech)
    """

    # -------------------------
    # AG News 전체 데이터 로드 (train split: 120k 샘플)
    # -------------------------
    ds = load_dataset('ag_news', split='train')

    rows = []
    # -------------------------
    # 0~3 라벨별로 균등 샘플링
    # -------------------------
    for lab in range(4):
        # 해당 라벨에 속하는 기사에서 n_per_class개 추출
        sub = ds.filter(lambda ex: ex['label'] == lab).select(range(n_per_class))
        for r in sub:
            rows.append({
                'text': r['text'],    # 전체 뉴스 기사 텍스트
                'label': r['label']   # 클래스 라벨
            })

    df = pd.DataFrame(rows)

    # 텍스트 길이 필터링 (너무 짧거나 긴 기사 제거)
    df['len'] = df['text'].str.len()
    df = df[(df['len'] >= 30) & (df['len'] <= 500)]

    # 중복 제거 및 인덱스 리셋
    df = df.drop_duplicates('text').drop(columns=['len']).reset_index(drop=True)

    return df



In [24]:
# 각 클래스에서 300개씩 샘플링하여 코퍼스 생성
corpus = load_news_corpus(300)

# 데이터셋 크기와 앞부분 샘플 확인
print(corpus.shape)
corpus.head(2)

Filter: 100%|██████████| 120000/120000 [00:00<00:00, 278898.69 examples/s]

(1170, 2)


,text,label
0,Venezuelans Vote Early in Referendum on Chavez...,0
1,S.Koreans Clash with Police on Iraq Troop Disp...,0


### 2. 임베딩 생성 및 캐시(npz)
- 소형 SBERT 모델 사용, CPU 배치 인코딩


### 2-1. (옵션) SBERT 미설치/다운로드 불가 시 TF‑IDF 검색으로 폴백
- 실습/현장 환경에서 `sentence-transformers` 설치나 모델 다운로드가 어려울 수 있음
- 이때 간단한 TF‑IDF 코사인 유사도 검색으로 대체하여 데모 지속


In [ ]:
# TF-IDF 폴백 구현 (SBERT 미사용 시)
import numpy as np
from typing import List, Tuple

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 전역 캐시 (코퍼스 로드 후 채움)
_VEC = None
_MAT = None


def tfidf_build(corpus_texts: List[str], max_features: int = 30000):
    """
    코퍼스를 TF-IDF로 변환하고 전역 캐시에 저장합니다.
    - normalize/L2는 사이킷런 내부에서 처리됨
    - 희소 행렬을 유지해 메모리 절약
    """
    global _VEC, _MAT
    _VEC = TfidfVectorizer(max_features=max_features, ngram_range=(1, 2))
    _MAT = _VEC.fit_transform(corpus_texts)
    return _VEC, _MAT


def tfidf_search(query: str, k: int = 5) -> List[Tuple[int, float, str]]:
    """
    TF-IDF 코사인 유사도로 top-k 문서를 반환합니다.
    반환: (문서인덱스, 유사도, 원문)
    """
    if _VEC is None or _MAT is None:
        raise RuntimeError('먼저 tfidf_build를 호출해 코퍼스를 준비하세요.')
    qv = _VEC.transform([query])
    sim = cosine_similarity(qv, _MAT)[0]  # shape: (n_docs,)
    top = np.argsort(-sim)[:k]
    return [(int(i), float(sim[i]), idx_text[i]) for i in top]



### 3-1. 하이브리드 검색(개념): TF‑IDF + SBERT
- 스파스(TF‑IDF)와 덴스(SBERT) 각각의 장점을 결합
- 간단 가이드: 두 유사도 점수를 0~1 정규화 후 \(\alpha\) 가중 합 → 상위 k


In [ ]:
# (선택) 간단 하이브리드 구현 — TF‑IDF + SBERT 가중합
# 전제: SBERT(E, idx_text)와 TF‑IDF(_VEC, _MAT, idx_text)가 준비되어 있다고 가정

def hybrid_search(query: str, k: int = 5, alpha: float = 0.5):
    """
    두 점수의 가중합으로 상위 k개 반환
    alpha: SBERT 가중(0~1), (1-alpha): TF‑IDF 가중
    """
    # SBERT 점수 (정규화 임베딩 기준: 코사인 유사도 = dot)
    qv = embed_corpus([query], batch_size=1)  # shape: (1, dim)
    sbert_scores = (E @ qv[0]).astype(float)  # shape: (n_docs,)

    # TF‑IDF 점수
    tfidf_scores = None
    if _VEC is not None and _MAT is not None:
        qv2 = _VEC.transform([query])
        tfidf_scores = cosine_similarity(qv2, _MAT)[0]
        tfidf_scores = np.asarray(tfidf_scores, dtype=float)

    # 경우의 수 처리
    if sbert_scores is None and tfidf_scores is None:
        raise RuntimeError('SBERT/TF‑IDF 어느 쪽도 준비되지 않았습니다.')
    if sbert_scores is None:
        scores = tfidf_scores
    elif tfidf_scores is None:
        scores = sbert_scores
    else:
        # 간단 min-max 정규화 후 가중합
        def norm01(x):
            x_min, x_max = float(np.min(x)), float(np.max(x))
            if x_max - x_min < 1e-12:
                return np.zeros_like(x)
            return (x - x_min) / (x_max - x_min)
        scores = alpha*norm01(sbert_scores) + (1-alpha)*norm01(tfidf_scores)

    top = np.argsort(-scores)[:k]
    return [(int(i), float(scores[i]), idx_text[i]) for i in top]



### 6. CPU 환경 팁(실무)
- SBERT: `all-MiniLM-L6-v2` 등 경량 모델, `device='cpu'`, `batch_size` 완만 조절
- 캐시: 임베딩 `npz` 저장/로드, 최초 1회만 생성
- 검색: `NearestNeighbors(metric='cosine', algorithm='brute')`는 소규모/데모에 충분
- 폴백: 설치/다운로드 불가 시 TF‑IDF 경로 사용, 가능하면 하이브리드로 보완
- 추론 속도: 실시간 질의 다량 처리 시 쿼리 배치 처리 또는 인덱스 라이브러리(FAISS/Annoy) 고려


### 7. 실습 과제(선택)
1) 질의 5개를 자체 정의하고, SBERT / TF‑IDF / 하이브리드 각각의 top‑3 결과를 비교해보세요.
2) 하이브리드의 \(\alpha\) 값을 {0.2, 0.5, 0.8}로 바꿔 결과 변화를 관찰하세요.
3) `compose_response` 템플릿을 수정해, 점수/출처 표시를 개선해보세요.


In [28]:
def embed_corpus(texts: list[str], model_name='sentence-transformers/all-MiniLM-L6-v2', batch_size=64):
    model = SentenceTransformer(model_name, device='cpu')
    outs = []
    for i in range(0, len(texts), batch_size):
        v = model.encode(texts[i:i+batch_size], batch_size=batch_size, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)
        outs.append(v)
    return np.vstack(outs)

cache_path = './data/embeddings_agnews.npz'
if os.path.exists(cache_path):
    dat = np.load(cache_path, allow_pickle=True)
    E = dat['E']; idx_text = dat['idx_text']
    idx_text = idx_text.tolist()
else:
    _t0 = time.perf_counter()
    E = embed_corpus(corpus['text'].tolist(), batch_size=64)
    dt = time.perf_counter()-_t0
    print('임베딩(sec):', round(dt,2))
    idx_text = corpus['text'].tolist()
    np.savez_compressed(cache_path, E=E, idx_text=np.array(idx_text, dtype=object))

E.shape


(1170, 384)

### 3. 최근접 탐색(코사인) 및 검색 함수


In [29]:
# sklearn NearestNeighbors로 코사인 기반 kNN 인덱스
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(E)


def search_topk(query: str, k: int = 5) -> list[tuple[int, float, str]]:
    # 쿼리 임베딩
    qv = embed_corpus([query], batch_size=1)
    dist, idx = nn.kneighbors(qv, n_neighbors=k, return_distance=True)
    # cosine distance → similarity = 1 - distance
    out = []
    for d, i in zip(dist[0], idx[0]):
        out.append((int(i), float(1.0 - d), idx_text[i]))
    return out

# 예시 검색
for i, (idx_i, sim, text) in enumerate(search_topk('New AI system beats benchmarks in vision tasks', k=5), 1):
    print(i, round(sim,3), text[:120]+'...')


1 0.346 Microsoft wants to improve your image New imaging software is making eyes at those squinty camera-phone pictures....
2 0.337 Sharp brings 3D to PCs, without the funny specs Firm brings tech already used in phones and laptops to desktops. Screen ...
3 0.315 The Eyes Are the Window to Hypertension The tiniest blood vessels of the eye can provide a glimpse that may warn of futu...
4 0.247 Were the judges by any chance French? Kosuke Kitajima #39;s technique may get a sharper eye from officials today in the ...
5 0.242 New NASA Supercomputer to Aid Theorists and Shuttle Engineers (SPACE.com) SPACE.com - NASA researchers have teamed up wi...


### 4. 미니 RAG 조합 함수
- retrieved 문서들을 템플릿으로 단순 요약/응답 생성


In [30]:
def compose_response(query: str, topk: list[tuple[int, float, str]], k: int = 3) -> str:
    parts = [f"Q: {query}", "\n[관련 문서 요약]"]
    for j, (idx_i, sim, text) in enumerate(topk[:k], 1):
        parts.append(f"- ({j}) 유사도 {sim:.2f}: {text[:160]}...")
    parts.append("\nA: 위 문서들을 참고하면, 질문과 직접적으로 연관된 핵심은 위 목록에서 확인 가능합니다.")
    return "\n".join(parts)

q = 'What is the impact of interest rate hikes on the stock market?'
res = search_topk(q, k=5)
print(compose_response(q, res, k=3))


Q: What is the impact of interest rate hikes on the stock market?

[관련 문서 요약]
- (1) 유사도 0.50: Treasuries Up, Rate Hike Still in Offing (Reuters) Reuters - U.S. Treasury debt made moderate gains\on Tuesday after a key reading of U.S. inflation proved soft...
- (2) 유사도 0.46: Election-Year Rate Hike Puzzles Some WASHINGTON - Going against conventional wisdom, the Federal Reserve is raising interest rates in an election year. And it i...
- (3) 유사도 0.43: South Korea lowers interest rates South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy....

A: 위 문서들을 참고하면, 질문과 직접적으로 연관된 핵심은 위 목록에서 확인 가능합니다.


### 5. 간단 데모
- 질의 입력 → top‑k 리스트와 응답 출력


In [31]:
queries = [
    'Latest advances in computer vision benchmarks',
    'Effects of monetary policy on markets',
    'Championship highlights of the season',
]
for q in queries:
    res = search_topk(q, k=5)
    print('='*60)
    print(compose_response(q, res, k=3))


Q: Latest advances in computer vision benchmarks

[관련 문서 요약]
- (1) 유사도 0.32: Microsoft wants to improve your image New imaging software is making eyes at those squinty camera-phone pictures....
- (2) 유사도 0.32: Sharp brings 3D to PCs, without the funny specs Firm brings tech already used in phones and laptops to desktops. Screen creates different pixel images for each ...
- (3) 유사도 0.32: Microsoft Upgrades Software for Digital Pictures  SEATTLE (Reuters) - Microsoft Corp. &lt;MSFT.O&gt; released on  Tuesday the latest version of its software for...

A: 위 문서들을 참고하면, 질문과 직접적으로 연관된 핵심은 위 목록에서 확인 가능합니다.
Q: Effects of monetary policy on markets

[관련 문서 요약]
- (1) 유사도 0.44: Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last wee...
- (2) 유사도 0.44: Treasuries Edge Ahead on Inflation Relief  NEW YORK (Reuters) - Treasury debt prices firmed on Tuesday  after a key reading of U.S. inflation pro

In [ ]:
# 2. TF-IDF 임베딩 재정의(설치 전제)
import numpy as np
from typing import List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 전역 캐시 (코퍼스 로드 후 채움)
_VEC = None
_MAT = None


def tfidf_build(corpus_texts: List[str], max_features: int = 30000):
    """
    코퍼스를 TF-IDF로 변환하고 전역 캐시에 저장합니다.
    - normalize/L2는 사이킷런 내부에서 처리됨
    - 희소 행렬을 유지해 메모리 절약
    """
    global _VEC, _MAT
    _VEC = TfidfVectorizer(max_features=max_features, ngram_range=(1, 2))
    _MAT = _VEC.fit_transform(corpus_texts)
    return _VEC, _MAT


def tfidf_search(query: str, k: int = 5) -> List[Tuple[int, float, str]]:
    """
    TF-IDF 코사인 유사도로 top-k 문서를 반환합니다.
    반환: (문서인덱스, 유사도, 원문)
    """
    if _VEC is None or _MAT is None:
        raise RuntimeError('먼저 tfidf_build를 호출해 코퍼스를 준비하세요.')
    qv = _VEC.transform([query])
    sim = cosine_similarity(qv, _MAT)[0]
    top = np.argsort(-sim)[:k]
    return [(int(i), float(sim[i]), idx_text[i]) for i in top]



In [ ]:
# 3. TF-IDF 벡터화 실행 (코퍼스 텍스트)
_ = tfidf_build(corpus['text'].tolist(), max_features=30000)
print(_MAT.shape)



In [ ]:
# 4. 세 가지 검색 방식 비교 데모
queries = [
    'Latest advances in computer vision benchmarks',
    'Effects of monetary policy on markets',
    'Championship highlights of the season',
]

for q in queries:
    print('='*80)
    print('Q:', q)
    # SBERT
    sbert_top = search_topk(q, k=5)
    print('\n[SBERT top-3]')
    for j, (i, sim, text) in enumerate(sbert_top[:3], 1):
        print(f' ({j}) {sim:.2f} | {text[:120]}...')
    
    # TF-IDF
    tfidf_top = tfidf_search(q, k=5)
    print('\n[TF-IDF top-3]')
    for j, (i, sim, text) in enumerate(tfidf_top[:3], 1):
        print(f' ({j}) {sim:.2f} | {text[:120]}...')
    
    # Hybrid
    hybrid_top = hybrid_search(q, k=5, alpha=0.5)
    print('\n[Hybrid top-3]')
    for j, (i, score, text) in enumerate(hybrid_top[:3], 1):
        print(f' ({j}) {score:.2f} | {text[:120]}...')

